#### Using data from okxe.vn

In [8]:
from urllib.request import urlopen
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
url = "https://webscraper.io/test-sites/tables"

In [10]:
html_code = urlopen(url).read().decode("utf-8")
soup = BeautifulSoup(html_code)

In [11]:
soup.find_all("h2")

[<h2>Semantically correct table with thead and tbody</h2>,
 <h2>Table without thead tag</h2>,
 <h2>Table with multiple header rows and an empty data row</h2>]

-----------------------------

In [12]:
brands_url = "https://api.okxe.vn/api/v2/brands?is_active=1"
models_url = "https://api.okxe.vn/api/v2/models?brand_id=1&is_active=1"
posts_url = "https://api.okxe.vn/api/v2/products?sort_of=product_new&page=1&per_page=256"
colors_url = "https://api.okxe.vn/api/v2/colors"

In [13]:
brands_data = requests.get(brands_url)
posts_data = requests.get(posts_url)
models_data = requests.get(models_url)
colors_data = requests.get(colors_url)

In [14]:
posts_data.json()

{'success': True,
 'result_code': 1,
 'result': 'Ok',
 'data': [{'used_status': 'used',
   'fuel': 'gasolin',
   'used_distance': 5000,
   'total_views': 0,
   'type': '2',
   'location_id': 2,
   'q_product_name': 'Yamaha Nouvo 5 125 2012',
   'price': 8800000,
   'seo_description': None,
   'model': {'is_active': 1,
    'sort_num': 99,
    'name': 'Nouvo 5',
    'id': 1083,
    'deleted_at': None,
    'brand_id': 40},
   'detail_model': {'user_admin_id': 0,
    'note': '',
    'is_active': 1,
    'fuel': 'Gasolin',
    'created_at': '2020-08-28 13:24:23',
    'model_id': 1083,
    'type': 'Xe số',
    'deleted_at': None,
    'updated_at': '2022-01-11 00:46:10',
    'engine': '125',
    'sort_num': 99,
    'name': '125',
    'old_id': None,
    'id': 1968,
    'gear': 'Manual'},
   'id': 1195862,
   'stock': 1,
   'brand': {'is_active': 1,
    'updated_at': '2019-09-25 16:22:56',
    'active_logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Yamaha.png',
    'sort_num': 1,
    'name': 

In [15]:
brands_data.json()['data']

[{'id': 1,
  'name': 'Aprillia',
  'logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Aprilia.png',
  'logo_normal': 'https://cdn.okxe.vn/moto/logo/inactive/2021/Aprilia.png',
  'sort_num': 99,
  'is_active': 1},
 {'id': 2,
  'name': 'Bajaj',
  'logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Bajaj.png',
  'logo_normal': 'https://cdn.okxe.vn/moto/logo/inactive/2021/Bajaj.png',
  'sort_num': 99,
  'is_active': 1},
 {'id': 3,
  'name': 'Benelli',
  'logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Benelli.png',
  'logo_normal': 'https://cdn.okxe.vn/moto/logo/inactive/2021/Benelli.png',
  'sort_num': 99,
  'is_active': 1},
 {'id': 4,
  'name': 'BMW',
  'logo': 'https://cdn.okxe.vn/moto/logo/active/2021/BMW.png',
  'logo_normal': 'https://cdn.okxe.vn/moto/logo/inactive/2021/BMW.png',
  'sort_num': 1,
  'is_active': 1},
 {'id': 5,
  'name': 'Brixton',
  'logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Brixtion.png',
  'logo_normal': 'https://cdn.okxe.vn/moto/logo/inactive/2021/Brixtion

In [16]:
models_data.json()['data']

[{'id': 1,
  'name': 'Caponord',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 3,
  'name': 'SR Motard',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 5,
  'name': 'RSV4',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 6,
  'name': 'Tuono V4',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 1159,
  'name': 'Dorsoduro',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 1163,
  'name': 'S4',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 1164,
  'name': 'Shiver',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'},
 {'id': 1166,
  'name': 'Tuono',
  'image': 'https://s3-ap-southeast-1.amazonaws.com/okxe.2019/moto/icon/bike_icon2.png'}]

In [17]:
posts_data.json()['data']

[{'used_status': 'used',
  'fuel': 'gasolin',
  'used_distance': 5000,
  'total_views': 0,
  'type': '2',
  'location_id': 2,
  'q_product_name': 'Yamaha Nouvo 5 125 2012',
  'price': 8800000,
  'seo_description': None,
  'model': {'is_active': 1,
   'sort_num': 99,
   'name': 'Nouvo 5',
   'id': 1083,
   'deleted_at': None,
   'brand_id': 40},
  'detail_model': {'user_admin_id': 0,
   'note': '',
   'is_active': 1,
   'fuel': 'Gasolin',
   'created_at': '2020-08-28 13:24:23',
   'model_id': 1083,
   'type': 'Xe số',
   'deleted_at': None,
   'updated_at': '2022-01-11 00:46:10',
   'engine': '125',
   'sort_num': 99,
   'name': '125',
   'old_id': None,
   'id': 1968,
   'gear': 'Manual'},
  'id': 1195862,
  'stock': 1,
  'brand': {'is_active': 1,
   'updated_at': '2019-09-25 16:22:56',
   'active_logo': 'https://cdn.okxe.vn/moto/logo/active/2021/Yamaha.png',
   'sort_num': 1,
   'name': 'Yamaha',
   'created_at': '2019-09-25 16:22:56',
   'inactive_logo': 'https://cdn.okxe.vn/moto/log

In [18]:
post = posts_data.json()['data'][0]

In [19]:
att_Vehicle_name = post['product_name']
att_Odometer = post['used_distance']
att_License_plate = post['license_plate']
att_Cubic_power = post['engine']
att_Manufacture_year = post['release_year']
att_City = post['location']['name']
att_ID_VehicleBrand = post['brand_id']
att_ID_VehicleLineup = post['model']['id']
att_Condition = post['used_status']
att_ID_VehicleType = post['type']
att_ID_Color = post['color'][0]['color_id']

In [20]:
att_Vehicle_name, att_Odometer, att_License_plate, att_Cubic_power, att_Manufacture_year, att_City, att_ID_VehicleBrand, att_ID_VehicleLineup, att_Condition, att_ID_VehicleType, att_ID_Color

('Yamaha Nouvo 5 125',
 5000,
 None,
 '125',
 201212,
 'Tp. Hồ Chí Minh',
 40,
 1083,
 'used',
 '2',
 16)

In [21]:
att_Title = post['q_product_name']
att_Content = post['description']
att_Pricetag = post['price']

att_Image = post['image']['medium']

In [22]:
att_Title, att_Content, att_Pricetag, att_Image

('Yamaha Nouvo 5 125 2012',
 'Cần b&aacute;n nouvo 5 như h&igrave;nh\nm&aacute;y m&oacute;c &ecirc;m ru\ngiấy tờ đầy đủ\ngi&aacute; 8tr8\nxem xe tại 135, đường số 51, t&acirc;n quy, quận 7',
 8800000,
 'https://cdn.okxe.vn/products/862/1195862/1195862_283586_IMG_20221205_134635_20221205014815_CAtMpdg3_m_400x400.jpg')

-----

In [23]:
brands_table = pd.DataFrame(data=brands_data.json()['data']).drop(labels=['logo_normal', 'sort_num', 'is_active'], axis=1)

In [24]:
brands_table

,id,name,logo
0,1,Aprillia,https://cdn.okxe.vn/moto/logo/active/2021/Apri...
1,2,Bajaj,https://cdn.okxe.vn/moto/logo/active/2021/Baja...
2,3,Benelli,https://cdn.okxe.vn/moto/logo/active/2021/Bene...
3,4,BMW,https://cdn.okxe.vn/moto/logo/active/2021/BMW.png
4,5,Brixton,https://cdn.okxe.vn/moto/logo/active/2021/Brix...
...,...,...,...
131,140,Victoria,https://cdn.okxe.vn/moto/logo/active/2021/Befo...
132,141,Taya Motor,https://cdn.okxe.vn/moto/logo/active/2021/Befo...
133,143,Lima,https://cdn.okxe.vn/moto/logo/active/2021/Befo...
134,144,Evgo,https://cdn.okxe.vn/moto/logo/active/2021/Befo...


In [25]:
total_data = []
for index, row in brands_table.iterrows():
   url = f"https://api.okxe.vn/api/v2/models?brand_id={row['id']}&is_active=1"
   data = requests.get(url).json()['data']
   for i in data:
      i['brand_id'] = row['id']
      total_data.append(i)

In [26]:
models_table = pd.DataFrame(data=total_data).drop(columns=['image'])

In [27]:
models_table

,id,name,brand_id
0,1,Caponord,1
1,3,SR Motard,1
2,5,RSV4,1
3,6,Tuono V4,1
4,1159,Dorsoduro,1
...,...,...,...
1240,1693,C,144
1241,1694,A,144
1242,1683,Enduro,145
1243,1684,Supermoto,145


In [28]:
colors_table = pd.DataFrame(data=colors_data.json()['data'])

In [29]:
colors_table

,id,name,color_code
0,1,white,None
1,2,ivory,None
2,3,black,None
3,4,silver,None
4,5,gray,None
5,6,gold,None
6,7,brown,None
7,8,yellow,None
8,9,orange,None
9,10,red,None


In [46]:
condition_data = [
   {'id': 1,
    'name': 'New'},
   {'id': 2,
    'name': 'Used'}
]
condition_table = pd.DataFrame(data=condition_data, index=None)

,id,name
0,1,New
1,2,Used


In [56]:
vehicletype_data = [
   {'id': 1,
    'name': 'Scooter'},
   {'id': 2,
    'name': 'Manual'},
   {'id': 3,
    'name': 'Motorbike'},
   {'id': 4,
    'name': 'Electric'}
]
vehicletype_table = pd.DataFrame(data=vehicletype_data, index=None)
vehicletype_table

,id,name
0,1,Scooter
1,2,Manual
2,3,Motorbike
3,4,Electric


In [30]:
num_request = 256
total_post = []
total_vehicleinfo = []

url = f"https://api.okxe.vn/api/v2/products?sort_of=product_new&page=1&per_page={num_request}&include=advertisement"
data = requests.get(url).json()['data']

Yamaha Latte
SYM Angel 110
Harley Davidson Forty-Eight Special
Harley Davidson Street 750
Yamaha Exciter 150
Honda Air Blade 125 FI
Yamaha Exciter 150
Honda LEAD 110
Yamaha Nozza Grande 125
Dkbike Romar 
Yamaha Nouvo 5 125
Honda Wave Alpha RSX 110 Fi
Honda Vision 110
Honda CBR 150 R
Yamaha R15 V2 150
Honda Vision 110
Honda Air Blade 125 FI
Honda Future 125
Honda PS 150i
Benelli TNT 125
Honda Air Blade 125 FI
Yamaha Exciter VVA 155
Honda Wave Alpha 110i
Honda Win 100
Honda Winner 150
Yamaha Exciter 150 RC
Yamaha Janus 2019
Honda Wave Alpha RSX 110 Fi
Honda Vision NSC110
Honda Air Blade 125 FI
Yamaha Sirius 110
SYM Shark 125
Honda Air Blade 125 FI
Honda Wave S 110
Honda Air Blade 125 FI
Honda MSX 125
Honda Wave Alpha 110i
Yamaha Janus 2019
Yamaha Nouvo 4 135
Honda MSX 125
Yamaha FZS FI
Honda Vario 150
Honda Winner 150
Honda Air Blade 110
Brixton BX 150 R
Honda Wave Alpha 110
Yamaha Janus 2019
Honda PCX 125
Yamaha MT 15
Ducati Mini 110
Honda LEAD 110
Yamaha Acruzo Deluxe 2016
Yamaha Jupit

In [ ]:
for j in range(num_request):
   item = data[j]
   vehicle_info = {
      'infoid': j + 1,
      'Vehicle_name': item['product_name'],
      'Odometer': item['used_distance'],
      'License_plate': item['license_plate'],
      'Cubic_power': item['engine'],
      'Manufacture_year': item['release_year'],
      'ID_VehicleBrand': item['brand_id'],
      'ID_VehicleLineup': item['model']['id'],
      'Condition': item['used_status'],
      'ID_VehicleType': item['type'],
      'ID_Color': item['color'][0]['color_id']
   }
   post = {
      'Title': item['q_product_name'],
      'Content': item['description'],
      'Pricetag': item['price'],
      'Image': item['image']['medium'],
      'City_name': item['location']['name'],
      'ID_VehicleInfo': j + 1
   }
   print(item['product_name'] or '###' + " - " + item['license_plate'] or '###')
   total_vehicleinfo.append(vehicle_info)
   total_post.append(post)

In [31]:
total_vehicleinfo

[{'infoid': 1,
  'Vehicle_name': 'Yamaha Latte',
  'Odometer': 10000,
  'License_plate': None,
  'Cubic_power': '125cc',
  'Manufacture_year': 202011,
  'ID_VehicleBrand': 40,
  'ID_VehicleLineup': 1711,
  'Condition': 'used',
  'ID_VehicleType': '1',
  'ID_Color': 10},
 {'infoid': 2,
  'Vehicle_name': 'SYM Angel 110',
  'Odometer': 100000,
  'License_plate': None,
  'Cubic_power': '110',
  'Manufacture_year': 200504,
  'ID_VehicleBrand': 35,
  'ID_VehicleLineup': 438,
  'Condition': 'used',
  'ID_VehicleType': '2',
  'ID_Color': 13},
 {'infoid': 3,
  'Vehicle_name': 'Harley Davidson Forty-Eight Special',
  'Odometer': 5000,
  'License_plate': None,
  'Cubic_power': '1200',
  'Manufacture_year': 201912,
  'ID_VehicleBrand': 13,
  'ID_VehicleLineup': 115,
  'Condition': 'used',
  'ID_VehicleType': '3',
  'ID_Color': 3},
 {'infoid': 4,
  'Vehicle_name': 'Harley Davidson Street 750',
  'Odometer': 5000,
  'License_plate': None,
  'Cubic_power': '750',
  'Manufacture_year': 201503,
  'ID_V

In [32]:
posts_table = pd.DataFrame(data=total_post)
vehicleinfo_table = pd.DataFrame(data=total_vehicleinfo)

In [33]:
posts_table

,Title,Content,Pricetag,Image,City_name,ID_VehicleInfo
0,Yamaha Latte 2020,Latte 2020 k&yacute; ngay chủ \nOdo chuẩn gần ...,35000000,https://cdn.okxe.vn/products/913/1113913/11139...,Tp. Hồ Chí Minh,1
1,SYM Angel 110 2005,Cầm Đồ Ho&agrave;ng Thắng Thanh L&yacute; Xe :...,4500000,https://cdn.okxe.vn/products/944/1102944/11029...,Tp. Hồ Chí Minh,2
2,Harley Davidson Forty-Eight Special 2019,XE MOTO PH&Acirc;N KHỐI LỚN CỰC HIẾM - Harley ...,435000000,https://cdn.okxe.vn/products/376/1069376/10693...,Tp. Hồ Chí Minh,3
3,Harley Davidson Street 750 2015,‼️ MOTOR HARLEY DAVIDSON STREET 750 CH&Iacute;...,178000000,https://cdn.okxe.vn/products/79/1012079/101207...,Tp. Hồ Chí Minh,4
4,Yamaha Exciter 150 2017,- Yamaha exciter 150\n- biển số th&agrave;nh p...,27500000,None,Tp. Hồ Chí Minh,5
...,...,...,...,...,...,...
251,Yamaha Exciter 150 2018,"M&aacute;y m&oacute;c nguy&ecirc;n bản, h&igra...",21800000,https://cdn.okxe.vn/products/605/1195605/11956...,Tp. Hà Nội,252
252,Benelli TNT 125 2019,"M&igrave;nh đang muốn đổi xe, l&ecirc;n cần b&...",15000000,https://cdn.okxe.vn/products/604/1195604/11956...,Tp. Hà Nội,253
253,Honda Wave Alpha 50 2011,"b&aacute;n wave 50 hq li&ecirc;n doanh, đẹp mớ...",7500000,https://cdn.okxe.vn/products/603/1195603/11956...,Tp. Hồ Chí Minh,254
254,Honda Air Blade 110 2011,M&igrave;nh đang cần b&aacute;n chiếc xe n&agr...,11000000,https://cdn.okxe.vn/products/602/1195602/11956...,Tp. Hà Nội,255


In [53]:
posts_table['City_name'].unique(), vehicleinfo_table['Condition'].unique()

(array(['Tp. Hồ Chí Minh', 'Tp. Hà Nội', 'Tỉnh Bình Dương',
        'Tỉnh Bắc Giang', 'Tp. Đà Nẵng'], dtype=object),
 array([2], dtype=object))

In [34]:
posts_table.isnull().sum()

Title             0
Content           0
Pricetag          0
Image             1
City_name         0
ID_VehicleInfo    0
dtype: int64

In [50]:
mask_new = vehicleinfo_table.Condition == 'new'
mask_used = vehicleinfo_table.Condition == 'used'
vehicleinfo_table.loc[mask_new, 'Condition'] = 1
vehicleinfo_table.loc[mask_used, 'Condition'] = 2

In [51]:
vehicleinfo_table

,infoid,Vehicle_name,Odometer,License_plate,Cubic_power,Manufacture_year,ID_VehicleBrand,ID_VehicleLineup,Condition,ID_VehicleType,ID_Color
0,1,Yamaha Latte,10000,None,125cc,202011,40,1711,2,1,10
1,2,SYM Angel 110,100000,None,110,200504,35,438,2,2,13
2,3,Harley Davidson Forty-Eight Special,5000,None,1200,201912,13,115,2,3,3
3,4,Harley Davidson Street 750,5000,None,750,201503,13,1231,2,3,3
4,5,Yamaha Exciter 150,30000,None,149.7,201706,40,535,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...
251,252,Yamaha Exciter 150,5000,None,149.7,201812,40,535,2,2,15
252,253,Benelli TNT 125,5000,None,"124,8",201901,3,15,2,2,3
253,254,Honda Wave Alpha 50,30000,None,100,201112,15,1055,2,2,8
254,255,Honda Air Blade 110,5000,None,108,201101,15,148,2,1,10


In [36]:
vehicleinfo_table.isnull().sum()

infoid                0
Vehicle_name          0
Odometer              0
License_plate       256
Cubic_power           4
Manufacture_year      0
ID_VehicleBrand       0
ID_VehicleLineup      0
Condition             0
ID_VehicleType        0
ID_Color              0
dtype: int64

In [52]:
brands_table.to_csv("VehicleBrand.csv", index=None)
models_table.to_csv("VehicleLineup.csv", index=None)
posts_table.to_csv("Post.csv", index=None)
vehicleinfo_table.to_csv("VehicleInfo.csv", index=None)
colors_table.to_csv('Color.csv', index=None)
condition_table.to_csv('VehicleCondition.csv', index=None)
vehicletype_table.to_csv('VehicleType.csv', index=None)